# Модуль typing

https://habr.com/ru/company/lamoda/blog/432656/

Создан для того, чтобы отслеживать правильность типов передаваемых аргументов в функции. Аннотации функций появились в Python 3.6, модуль typing - в 3.7.

Аннотации типов призваны помочь программисту быстрее понять сигнатуру функции, но не предотвращают ошибок в передаваемых данных. Также они поддерживаются PyCharm'ом и линтерами.

In [ ]:
def upper(s: str) -> str:
    print("entered")
    return s.upper()

upper(12342)

Установим популярный статический анализатор mypy:

In [ ]:
!pip install mypy

Код из предыдущей ячейки записан в файле 07/typing1.py. Попробуем прогнать его через mypy:

In [ ]:
!mypy 07/typing1.py

### Алиасы типов

Благодаря возможностям библиотеки typing, можно часто используемые составные типы записывать в переменные:

In [ ]:
from typing import List
Vector = List[float]

def scale(scalar: float, vector: Vector) -> Vector:
    return [scalar * num for num in vector]


scale(2, [1, -4.2, 5.2])

## Структуры типов модуля typing

- `Any` - произвольный тип
- `List[int]` - список, который содержит только один тип данных int
- `Tuple[int, str]` - кортеж, который может содержать несколько типов данных
- `Union[str, bytes]` - можно подавать либо строку, либо bytes
- `Callable[[int, int], float]` - вызываемый объект, который на вход принимает два аргумента int, а возвращает float
- `Iterable[T]` - Iterable со значениями типа T
- `Mapping[K, V]`, `Dict[K, V]` - словарь с ключами типа K и значениями типа V
- `Awaitable[T_co]`
- `Type[T]` - тип/класс


- `Optional[T]` - показывает, что переменная может быть None

### Optional

По умолчанию аннотированный тип не может быть None:

In [ ]:
from typing import Optional

amount: int
amount = None  # Incompatible types in assignment (expression has type "None", variable has type "int")

price: Optional[int]
price = None

In [ ]:
!mypy -c "amount: int = None"

In [ ]:
!mypy -c "from typing import Optional; amount: Optional[int] = None"

### Any

Если мы берем на себя ручную обработку типов, можем аннотировать переменную классом Any, тогда она сможет принимать любые значения

In [ ]:
from typing import Any

amount: Any
amount = 1
amount = "Some"
amount = None

In [ ]:
!mypy -c "from typing import Any; amount: Any; amount = 1; amount = 'Some'; amount = None"

### Union

Предназначен для случаев, когда можно использовать только некоторые типы

In [ ]:
from typing import Union, Dict

def some_func(arg: Union[Dict[str, str], str]) -> int:
    return len(arg)


some_func({"a": "b"})
some_func("abc")
some_func({"a": 1})

In [ ]:
!mypy 07/typing2.py

## Generic-типы

Иногда нужно просто указать, что данные должны быть однотипными, без жесткой фиксации типов. Для этого используется TypeVar:

In [ ]:
from typing import TypeVar, Generic

T = TypeVar("T")

class LinkedList(Generic[T]):
    data: T
    next: "LinkedList[T]"

    def __init__(self, data: T):
        self.data = data

head_int: LinkedList[int] = LinkedList(1)
head_int.next = LinkedList(2)
head_int.next = 2  # error: Incompatible types in assignment (expression has type "int", variable has type "LinkedList[int]")
head_int.data += 1
head_int.data.replace("0", "1")  # error: "int" has no attribute "replace"

head_str: LinkedList[str] = LinkedList("1")
head_str.data.replace("0", "1")

head_str = LinkedList[str](1)  # error: Argument 1 to "LinkedList" has incompatible type "int"; expected "str"

In [ ]:
!mypy 07/typing3.py

Еще один пример - пара объектов для любых конкретных типов:

In [ ]:
from typing import TypeVar, Generic

K = TypeVar('K')
V = TypeVar('V')

class Pair(Generic[K, V]):
    def __init__(self, key: K, value: V):
        self._key = key
        self._value = value

    @property
    def key(self) -> K:
        return self._key

    @property
    def value(self) -> V:
        return self._value


class IntPair(Pair[int, int]):
    pass

p = IntPair("1", "2")

## Cast

Иногда статический анализатор не может однозначно определить тип переменной. Чтобы показать анализатору, что возвращается действительно заявленный тип, можно в коде использовать функцию cast.

In [ ]:
from typing import List, cast

def find_first_str(a: List[object]) -> str:
    index = next(i for i, x in enumerate(a) if isinstance(x, str))
    return cast(str, a[index])

Это полезно для декораторов, поскольку анализатору может быть непонятно, что представляет собой обобщенный wrapper:

In [ ]:
MyCallable = TypeVar("MyCallable", bound=Callable)

def logged(func: MyCallable) -> MyCallable:
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(func.__name__, args, kwargs)
        return func(*args, **kwargs)

    return cast(MyCallable, wrapper)

@logged
def mysum(a: int, b: int) -> int:
    return a + b

mysum(a=1)  # error: Missing positional argument "b" in call to "mysum"